In [ ]:
!pip install mace-torch
!pip install cuequivariance cuequivariance-torch cuequivariance-ops-torch-cu12

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import sys
sys.path.append('/content/drive/My Drive/colab_temp/mace_fine-tuning_expts/scripts')

In [ ]:
import utils

In [ ]:
%cd "drive/MyDrive/colab_temp/mace_fine-tuning_expts"

In [ ]:
from ase.units import create_units
units_2006 = create_units('2006')

# The values I back-computed with 2006 CODATA values
E0_Hf_Ry = -112.41607411999959316330
E0_O_Ry = -32.6203517400001970321

# Convert to eV using ASE units
E0_Hf_eV = E0_Hf_Ry * units_2006['Ry']  # Rydberg to eV conversion
E0_O_eV = E0_O_Ry * units_2006['Ry']

new_E0s = {8:  E0_O_eV,
           72: E0_Hf_eV}

In [ ]:
%%writefile configs/small_ft-HfOx_v0.2.yml

model: 'MACE'
foundation_model: 'small'
multiheads_finetuning: False
train_file: 'data/HfOx_Hf_trainval.xyz'
valid_fraction: 0.1
test_file: 'data/HfOx_Hf_test.xyz'
energy_key: "REF_energy"
forces_key: "REF_forces"
E0s: {8: -443.8224565134432, 72: -1529.4984727695407}
name: "small_ft-HfOx_v0.2"
model_dir: "models/small_HfOx-ft_2"
log_dir: "models/small_HfOx-ft_2"
results_dir: "models/small_HfOx-ft_2"
checkpoints_dir: "models/small_HfOx-ft_2"
device: cuda
batch_size: 5
max_num_epochs: 500
lr: 0.01
swa: True
seed: 123
stress_weight: 0.0
forces_weight: 50.0
energy_weight: 1.0

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path, "--plot", "true", "--plot_frequency", "10"]
    mace_run_train_main()

In [ ]:
train_mace("configs/small_ft-HfOx_v0.2.yml")

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from mace.cli.plot_train import main as mace_plot_train_main
import sys
import logging

def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]
    mace_run_train_main()